<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#성별-리뷰를-얼마나-남기는지" data-toc-modified-id="성별-리뷰를-얼마나-남기는지-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>성별 리뷰를 얼마나 남기는지</a></span></li><li><span><a href="#성별-피부타입" data-toc-modified-id="성별-피부타입-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>성별 피부타입</a></span></li><li><span><a href="#성별-평점-비율" data-toc-modified-id="성별-평점-비율-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>성별 평점 비율</a></span></li><li><span><a href="#성별-리뷰를-남기는-브랜드/상품/카테고리/유형" data-toc-modified-id="성별-리뷰를-남기는-브랜드/상품/카테고리/유형-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>성별 리뷰를 남기는 브랜드/상품/카테고리/유형</a></span></li></ul></div>

In [8]:
import pandas as pd

import pyecharts as pye
import matplotlib.pyplot as plt
import seaborn as sns

import os
os.listdir('../dataset')

['glowpick_products.csv',
 'glowpick_reviews.csv',
 'oliveyoung_product_info.csv']

# Load Data

In [125]:
glowpick_products = pd.read_csv('../dataset/glowpick_products.csv')
print('glowpick_products shape: ',glowpick_products.shape)

glowpick_products shape:  (1183, 8)


In [126]:
glowpick_products.head()

,brand,product,vol_price,rate,nb_reviews,sales_rank,product_url,category
0,이니스프리(innisfree),포레스트 포맨 모이스처 스킨,"180ml19,000원",4.07,(81),1,/product/7212,스킨
1,비오템옴므,아쿠아파워 토너,"200ml54,000원",4.09,(65),2,/product/4108,스킨
2,키엘 (Kiehl's),칼렌듈라 허벌 엑스트렉트 토너,"250ml59,000원",3.82,(730),3,/product/11071,스킨
3,반트 36.5 (VANT 36.5),포 맨 올 인 원 스킨,"120ml35,000원",4.29,(31),4,/product/21577,스킨
4,프리메라,맨 오가니언스 트리트먼트 워터,"180ml30,000원",4.38,(26),5,/product/13159,스킨


In [70]:
reviews = pd.read_csv('../dataset/glowpick_reviews.csv')
print('reviews shape: ',reviews.shape)

reviews shape:  (5125, 7)


In [71]:
reviews.head()

,date,user_id,sex,age_skin_type,rate,content,product_url
0,28일 전,minseo6666,f,16세 · 복합성 ·,good,아빠 사드렸는데 저아해여,/product/7212
1,2개월 전,피부에관심있는남자,m,20세 · 복합성 ·,good,향이 좋아요 근데 좀 제 피부가 민감해서,/product/7212
2,3개월 전,타라타라,f,23세 · 복합성 ·,good,가격이 저렴하고 냄새가 안 쎄서 좋아요. 👍,/product/7212
3,4개월 전,흐시너,f,18세 · 복합성 ·,good,선물용으로 사서 선물했는데 나름 만족했는지 매일 바른다네요;,/product/7212
4,4개월 전,홍끼루,f,32세 · 복합성 ·,good,향이 청량해서 좋아요.,/product/7212


# Preprocessing

TODO: 588개는 어디서 생긴걸까

In [136]:
# 리뷰가 있는 상품만 join
glowpick = pd.merge(reviews, glowpick_products, on='product_url', how='inner')
print('glowpick.shape: ',glowpick.shape)

glowpick.shape:  (5713, 16)


In [72]:
# 성별명 변환
reviews['sex'] = reviews.sex.map({'f':'여성','m':'남성'})

# 나이와 피부타입 분리
age_skin_df = reviews.age_skin_type.str.replace('·','').str.split('  ',expand=True)
age_skin_df.columns = ['age','skin_type']
age_skin_df['skin_type'] = age_skin_df.skin_type.str.strip()
reviews = pd.concat([reviews, age_skin_df], axis=1)

# 성별 리뷰를 얼마나 남기는지

In [160]:
# 성별 리뷰 수 카운트
sex_cnt = reviews.sex.value_counts()

# make pie charts
sex_pie = pye.Pie(width=500, height=300)
sex_pie.add(' ',sex_cnt.index, sex_cnt.tolist(), 
            is_label_show=True,
            radius=[20, 50])
sex_pie.render()

# 성별 피부타입

In [74]:
f_skin_type_cnt = reviews[reviews.sex=='여성'].skin_type.value_counts()
m_skin_type_cnt = reviews[reviews.sex=='남성'].skin_type.value_counts()

In [94]:
skin_pie = pye.Pie("성별에 따른 피부타입 분포 [왼쪽: 여성, 오른쪽: 남성]", title_pos='center', width=850)
skin_pie.add("여성", f_skin_type_cnt.index, f_skin_type_cnt.tolist(), 
             center=[25, 50], radius=[20, 50], rosetype='radius', 
             is_random=True, is_label_show=True)
skin_pie.add("남성", m_skin_type_cnt.index, m_skin_type_cnt.tolist(), 
             center=[75, 50], radius=[20, 50], rosetype='radius', 
             is_random=True, is_label_show=True, is_legend_show=False)
skin_pie

# 성별 평점 비율

In [95]:
f_rate_cnt = reviews[reviews.sex=='여성'].rate.value_counts()
m_rate_cnt = reviews[reviews.sex=='남성'].rate.value_counts()

In [96]:
skin_pie = pye.Pie("성별에 따른 평점 분포 [왼쪽: 여성, 오른쪽: 남성]", title_pos='center', width=850)
skin_pie.add("여성", f_rate_cnt.index, f_rate_cnt.tolist(), 
             center=[25, 50], radius=[20, 50], rosetype='radius', 
             is_random=True, is_label_show=True)
skin_pie.add("남성", m_rate_cnt.index, m_rate_cnt.tolist(), 
             center=[75, 50], radius=[20, 50], rosetype='radius', 
             is_random=True, is_label_show=True, is_legend_show=False)
skin_pie

# 성별 리뷰를 남기는 브랜드/상품/카테고리/유형

제품 10개이상인 브랜드만 비교

In [152]:
brand_cnt = glowpick.brand.value_counts().reset_index()
cnt_median = brand_cnt.describe().loc['50%'][0]
brand_cnt_cut = brand_cnt[brand_cnt['brand'] > cnt_median]
brand_cut = glowpick[glowpick.brand.isin(brand_cnt_cut['index'])]

f_brand_cut = brand_cut[brand_cut.sex=='여성'].brand.value_counts()
m_brand_cut = brand_cut[brand_cut.sex=='남성'].brand.value_counts()

In [165]:
pd.read_csv('../dataset/oliveyoung_product_info.csv').category.unique()

array(['스킨케어', '바디케어', '헤어케어', '메이크업', '향수/탈취', '쉐이빙/잡화'], dtype=object)

In [162]:
glowpick_products.category.unique()

array(['스킨', '로션', '에센스', '크림', '클렌징', '스크럽/필링', '쉐이빙', '애프터쉐이브', '마스크/팩',
       '메이크업', '선케어', '헤어/바디', '헤어스타일링'], dtype=object)

In [161]:
glowpick_products[glowpick_products.category=='향수']

,brand,product,vol_price,rate,nb_reviews,sales_rank,product_url,category
